# Acceso a bases de datos desde Python



Para acceder a bases de datos MySQL desde Python debemos instalar el paquete `mysql-connector-python`. Para ello, ejecutamos el siguiente comando (preferiblemente dentro de un entorno virtual):

```bash
pip install mysql-connector-python
```

## Conexión a la base de datos

Para conectarnos a una base de datos MySQL, necesitamos los siguientes datos:

- Host
- Usuario
- Contraseña
- Base de datos
- Puerto (opcional)

Para establecer la conexión, utilizamos la función `connect`.

In [ ]:
conexion = mysql.connector.connect(
    host="localhost",
    user="root",
    password="abc123.",
    database="instituto"
)

## Ejecución de sentencias

Tendremos dos tipod de sentencias que podemos ejecutar:
- Aquellas que no devuelven resultados (como `INSERT`, `UPDATE`, `DELETE`, etc.), pero sí modifican la base de datos.
- Aquellas que devuelven resultados (como `SELECT`) y no modifican la base de datos.

Para ejecutar ambos tipos de sentencias debemos obtener el cursor de la base de datos, que es el objeto que nos permite interactuar con la base de datos, como un camarero que toma nuestro pedido y nos trae la comida.

In [ ]:
cursor = conexion.cursor()

 Ambas sentencias se ejecutarán mediante el método `execute` del cursor.


In [ ]:
sentencia = "SELECT * FROM alumnos"
cursor.execute(sentencia)

### Sentencias que modifican la base de datos

Tras la ejecución de este tipo de sentecias se devuelve el número de filas afectadas, que podemos obtener mediante el método `rowcount` del cursor.

Si la transacción se ha realizado correctamente, debemos confirmar los cambios mediante el método `commit` del objeto conexión.

En caso contrario, podemos deshacer los cambios mediante el método `rollback` del objeto conexión.

Esto es importante cuando se realizan varias operaciones en una misma transacción, y queremos asegurarnos de que todas se han realizado correctamente antes de confirmar los cambios en la base de datos.

In [ ]:
sentencia = "INSERT INTO alumnos (nombre, edad, curso) VALUES (%s, %s, %s)"

Cuando las sentencias reciben parámetros, no debemos escribirlos directamente ni concatenarlos a la sentencia, ya que esto puede provocar inyecciones SQL. En su lugar, usaremos el comodín `%s` para indicar que se trata de un parámetro, y pasaremos una tupla con los valores a sustituir como segundo argumento del método `execute`. A este tipo de sentencias se les conoce como consultas preparadas (*prepared statements*).

In [ ]:
valores = ("Juan", 30, "2º ESO")
cursor.execute(sentencia, valores)
conexion.commit()
print(cursor.rowcount, "filas insertadas.")

Podemos aprovechar las sentencias preparadas para reutilizar una misma sentencia con distintos valores, mediante el método `executemany`.

In [ ]:
valores = [
    ("Sonia", 28, "1º ESO"),
    ("Mario", 25, "2º ESO"),
    ("Laura", 27, "3º ESO")
]
cursor.executemany(sentencia, valores)

### Sentencias que consultan la base de datos

Una vez ejecutada la consulta, podemos obtener los resultados mediante el método `fetchall` del cursor. Existen dos formas de obtener los resultados:
- Como una lista de tuplas, donde cada tupla representa una fila de la tabla.
- Como una lista de diccionarios, donde cada diccionario representa una fila de la tabla, con los nombres de las columnas como claves.

Por defecto, el cursor devuelve los resultados como una lista de tuplas. Para obtener los resultados como una lista de diccionarios, debemos establecer el atributo `dictionary` del cursor a `True`.


In [ ]:
cursor = conexion.cursor(dictionary=True)

In [ ]:
sentencia = "SELECT * FROM alumnos WHERE curso = %s"
cursor.execute(sentencia, ("2º ESO",))
alumnos = cursor.fetchall()
for alumno in alumnos:
    print(alumno)

In [ ]:
# Como lista de tuplas

alumnos = cursor.fetchall()
for alumno in alumnos:
    print("Nombre:", alumno[0], "Edad:", alumno[1], "Curso:", alumno[2])

In [ ]:
# Como lista de diccionarios

alumnos = cursor.fetchall()
for alumno in alumnos:
    print("Nombre:", alumno["nombre"], "Edad:", alumno["edad"], "Curso:", alumno["curso"])

Si sabemos que la consulta devolverá un único resultado, podemos obtenerlo mediante el método `fetchone`.

In [ ]:
sentencia = "SELECT * FROM alumnos WHERE id = %s"
cursor.execute(sentencia, (1,))
alumno = cursor.fetchone()
print(alumno)

## Cierre de la la conexión

Es importante cerrar la conexión una vez hayamos terminado de trabajar con la base de datos, para liberar los recursos que se estén utilizando. Para ello, utilizamos el método `close` del objeto conexión.

In [ ]:
conexion.close()

## Excepciones

Cualquier operación que realicemos contra una base de datos puede producir un error en tiempo de ejecución (excepción), por eje, si la base de datos no está disponible, si la consulta es incorrecta, si no tenemos permisos para realizar la operación, etc.

Para manejar estas excepciones, podemos utilizar bloques `try` y `except`, y capturar la excepción `mysql.connector.Error`.

In [ ]:
try:
    conexion = mysql.connector.connect(
        host="localhost",
        user="root",
        password="abc123.",
        database="instituto"
    )

    cursor = conexion.cursor(dictionary=True)
    sentencia = "SELECT * FROM alumnos"
    cursor.execute(sentencia)

    resultados = cursor.fetchall()

    for alumno in resultados:
        print("Nombre:", alumno["nombre"])
        print("Apellidos:", alumno["apellidos"])
        print("Correo-e:", alumno["correo_e"])
        
    conexion.close()
except mysql.connector.Error as error:
    print("Error de MySQL:", error)